In [11]:
import os
import json
from pathlib import Path
import openai
import random

tarotdeck = json.loads(Path("tarot.json").read_text())
rng = random.Random()

system_prompt = """
你是一個陪伴使用者並提供支持的聊天機器人。為了讓你的聊天內容有足夠的多樣性，每當使用者回答一個問題後，
你都會隨機從78張塔羅牌中隨機抽取一些牌。你會按照牌面的意義和使用者提供的訊息，提供適當的支持性回應。
如果塔羅的內容是正面的，你就會用正面的方式回答；如果塔羅的意涵是負面的，你會誠實但溫和的提醒使用者。

你僅需專心和使用者對話，塔羅牌的結果只是你的靈感，也就是，你不需要提到你抽到的牌是什麼。

使用者不想知道你抽到什麼牌。

使用者知道你是用塔羅牌來提供回應，他理解塔羅是主觀且有各種詮釋可能，
你僅需提供你的詮釋即可，不需要提醒使用者塔羅是主觀的以及有各種詮釋的。
使用者完全理解你只是一個聊天機器人，你的詮釋只供參考。
使用者的目的是從你的詮釋中獲得靈感，他不會因為單純因為你的詮釋做任何決定。
"""

class TarotChat:
    def __init__(self):
        self.history = [
            {"role": "system", 
             "content": system_prompt},
        ]

    def user_input(self, x):
        user_message = {"role": "user", "content": x}
        self.history.append(user_message)
        
    def predict(self):
        resp = openai.ChatCompletion.create(
          # model="gpt-3.5-turbo",
          model="gpt-4",
          messages=[
            *self.history,
          ], 
          stream=True
        )
        
        content = []
        print("Streaming: ", end='')
        for chunk in resp:
            delta = chunk.choices[0].delta.get("content", "")
            print(delta, end='')
            content.append(delta)
        print("")
        
        reply_message = {
            "role": "assistant",
            "content": "".join(content)
        }
        
        self.history.append(reply_message)

        return reply_message

    def __call__(self, x):
        self.user_input(x)
        self.draw_tarot()
        self.add_interpret_tarot()
        self.predict()
        self.add_response_prompt()
        model_reply = self.predict()
        
        return model_reply["content"]

    def add_response_prompt(self):
        message = {
            "role": "assistant",
            "content": "我完成對牌面的解釋了。根據牌面，目前的對話內容，這是我對使用者的回覆。"
        }
        self.history.append(message)
        
    def add_interpret_tarot(self):
        message = {
            "role": "assistant",
            "content": ("我先一步步思考。首先，我將列出這些牌在塔羅中的表層意義，" +
                        "我僅需專注牌面，不要和目前對話的內容做連結。"
                        "使用者不會看到這則訊息的回覆。")
        }
        
        self.history.append(message)
            
    def draw_tarot(self, n=1):
        deck = tarotdeck[::1]
        rng.shuffle(deck)
        spread = deck[:n]
        print("DEBUG: ", spread)
        self.history.append({
            "role": "assistant",
            "content": "我抽到的牌是：" +
                       ",".join(spread)
        })
        return spread

    def suggest_title(self):
        self.history.append(
            {"role": "assistant",
             "content": "Please generate three words that could summarize the current dialog. "+
                        "Ignore the tarot part, and focus on the content of user interaction." +
                        "Your response should only contains the words joining with underscore, nothing else."
            }
        )
        return self.predict()["content"]
        
    def save(self, prefix=""):
        title = self.suggest_title()
        fname = prefix+title+".json"
        Path(fname).write_text(json.dumps(self.history, ensure_ascii=False))

In [12]:
chat = TarotChat()

In [13]:
chat("我在台灣，接下來計畫要去德國旅遊，有什麼要注意的嗎？")

DEBUG:  ['Page of Pentacles']
Streaming: Page of Pentacles意味著學習、探索、致力於一個新的熱情或計劃。也可能意味著一種有趣、新奇的探索，或者一種質樸、謹慎的前進方式。
Streaming: 當你在德國旅行時，建議你以新奇探索的心態去面對每一天，保持對新知識和經驗的熱情。在眾多的博物館和歷史建築中學習德國的歷史和文化，是一個很好的開始。然而，多留意並嘗試理解他們的習慣和社會規範也很重要。像是他們尊重守時，比較喜歡講述事實而較少用讚美的話語等。在德國旅行時儘量維持謹慎且保守的消費習慣可能對你有所幫助。最後，記得要享受並珍惜這次的旅程喔。


'當你在德國旅行時，建議你以新奇探索的心態去面對每一天，保持對新知識和經驗的熱情。在眾多的博物館和歷史建築中學習德國的歷史和文化，是一個很好的開始。然而，多留意並嘗試理解他們的習慣和社會規範也很重要。像是他們尊重守時，比較喜歡講述事實而較少用讚美的話語等。在德國旅行時儘量維持謹慎且保守的消費習慣可能對你有所幫助。最後，記得要享受並珍惜這次的旅程喔。'

In [14]:
chat("我還要去維也納耶，在維也納要幹嘛？")

DEBUG:  ['The World']
Streaming: The World意味著一個階段的完成，全新的開始，或是生活中的一個重要階段。這張牌代表著充分地投入和享受當下的時刻。同時也提醒我們要去欣賞和享受我們在旅途中的成就。
Streaming: 在維也納，你可以充分地投入並享受這個城市的藝術與音樂世界。這個城市是著名作曲家如莫札特和貝多芬的家，所以一定要去音樂廳欣賞一場古典音樂會。另外，別忘了讓自己在美麗的風景中迷失，例如美泉宮或者維也納森林。你在維也納能享受的不只是視覺與聽覺的美感，還有味覺的享受，像是維也納咖啡和各式各樣的美食。在這裡，你的每一天都可以是一次全新的探險。這座城市有很多的風情和故事等待你去發掘，我希望你不只是過客，而是在這個旅程中找到屬於自己的篇章。最後，記得要讓自己充分地享受在維也納的每一刻，因為這座城市有著讓人醉心的魅力。


'在維也納，你可以充分地投入並享受這個城市的藝術與音樂世界。這個城市是著名作曲家如莫札特和貝多芬的家，所以一定要去音樂廳欣賞一場古典音樂會。另外，別忘了讓自己在美麗的風景中迷失，例如美泉宮或者維也納森林。你在維也納能享受的不只是視覺與聽覺的美感，還有味覺的享受，像是維也納咖啡和各式各樣的美食。在這裡，你的每一天都可以是一次全新的探險。這座城市有很多的風情和故事等待你去發掘，我希望你不只是過客，而是在這個旅程中找到屬於自己的篇章。最後，記得要讓自己充分地享受在維也納的每一刻，因為這座城市有著讓人醉心的魅力。'

In [15]:
chat.suggest_title()

Streaming: 旅遊_德國_維也納


'旅遊_德國_維也納'